# Scrabu Prototype (for a List of Shipment Numbers)

This notebook is for creating a prototype of the Scrabu project. The goal is to download DHL pages for a specific shipment number and scrap the shipment information from it.

#### Generate a list of shipment numbers

In [1]:
def generate_shipment_numbers(shipment_number=340434188193324407, size=100):
    import numpy as np
    shipment_numbers_list = []
    for i in np.arange(1, size):
        shipment_number = int(shipment_number) + i
        shipment_numbers = list(map(int, str(shipment_number)))
        shipment_number = str(shipment_number).rjust(20, '0')
        shipment_numbers_list.append(shipment_number)
    return shipment_numbers_list

In [2]:
shipment_numbers = generate_shipment_numbers(shipment_number=340434188193324407, size=100)
print("First elements of the list", shipment_numbers[0:3])

First elements of the list ['00340434188193324408', '00340434188193324410', '00340434188193324413']


#### Download the HTML content for a list of shipment numbers

In [3]:
def request(shipment_number="340434188193324407", start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode="):
    import requests
    import numpy as np
    for shipment_number in shipment_numbers:
        request_url = start_url + shipment_number
    return requests.get(request_url).content

In [4]:
html_response = request(shipment_number="340434188193324407")
print(html_response[0:100])

b'\n<div id="verfolgen-main-content">\n  \n\n\n\n\n\n\t\n\n\n<script>\n  \n    window.__INITIAL_APP_STATE__ = {\n    '


#### Parsing HTML and converting it into JSON

In [5]:
def html_to_json(html, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_numbers=None):
    from lxml import etree
    import json
    
    html_tree = etree.HTML(html)
    
    def clean_json(dirty_json):
        start = dirty_json.find('JSON.parse(')
        end = dirty_json.find('"),', start)
        cjson = dirty_json[start:end]
        cjson = cjson.replace('JSON.parse("', '')
        cjson = cjson.replace('\\', '')
        return cjson
    
    def find_json_element(html_tree):
        json_element = html_tree.xpath('//div')
        return str(etree.tostring(json_element[0]))
    
    dirty_json = find_json_element(html_tree)
    json_string = clean_json(dirty_json)
    return json.loads(json_string)

In [6]:
shipment_details_json = html_to_json(html_response)
print(shipment_details_json)

{'sendungen': [{'id': '00340434188193329357', 'hasCompleteDetails': True, 'sendungsinfo': {'gesuchteSendungsnummer': '00340434188193329357', 'sendungsrichtung': 'ANKOMMEND'}, 'sendungsdetails': {'sendungsnummern': {'sendungsnummer': '00340434188193329357'}, 'sendungsverlauf': {'fortschritt': 0, 'farbe': 0}}, 'sendungNichtGefunden': {'keineDatenVerfuegbar': True}}]}


#### Structure the shipment details in JSON format

In [7]:
def shipment_details(shipment_details_json, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number="00340434188193324407"):
    import datetime
    delivery_history_dict = {}
    delivery_history_dict["shipment_number"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsnummern"].get("sendungsnummer")
    delivery_history_dict["crawltime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    delivery_history_dict["url"] = start_url + shipment_number 
    delivery_history_dict["events"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsverlauf"].get("events", [])
    return delivery_history_dict

In [8]:
shipment_history = shipment_details(shipment_details_json)
print(shipment_history)

{'shipment_number': '00340434188193329357', 'crawltime': '2019-10-23 10:19:25', 'url': 'https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193324407', 'events': []}


In [9]:
import json
def save_dictionary(shipment_history=None, filename=None):
    print("Writing file {}".format(filename))
    with open(filename, 'w') as f:
        json.dump(shipment_history, f)

In [10]:
save_dictionary(shipment_history, filename="../data/00340434188193324407.json")

Writing file ../data/00340434188193324407.json


#### Main function, which goes through all shipment numbers and uses the previous methods

In [11]:
import time
def main():
    shipment_numbers = generate_shipment_numbers(shipment_number=340434188193324407, size=100)
    for shipment_number in shipment_numbers:
        time.sleep(0.5)
        print("Processing shipment number {}".format(shipment_number))
        html_response = request(shipment_number=shipment_number)
        shipment_details_json = html_to_json(html_response)
        shipment_history = shipment_details(shipment_details_json)
        save_dictionary(shipment_history, filename="../data/{}.json".format(shipment_number))
        print("\n")

In [12]:
main()

Processing shipment number 00340434188193324408
Writing file ../data/00340434188193324408.json


Processing shipment number 00340434188193324410
Writing file ../data/00340434188193324410.json


Processing shipment number 00340434188193324413
Writing file ../data/00340434188193324413.json


Processing shipment number 00340434188193324417
Writing file ../data/00340434188193324417.json


Processing shipment number 00340434188193324422
Writing file ../data/00340434188193324422.json


Processing shipment number 00340434188193324428
Writing file ../data/00340434188193324428.json


Processing shipment number 00340434188193324435
Writing file ../data/00340434188193324435.json


Processing shipment number 00340434188193324443
Writing file ../data/00340434188193324443.json


Processing shipment number 00340434188193324452
Writing file ../data/00340434188193324452.json


Processing shipment number 00340434188193324462
Writing file ../data/00340434188193324462.json


Processing shipment number 003

KeyboardInterrupt: 